# Metrics

> Metric tracking and analysis tools

In [ ]:
#| default_exp metrics

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np
from numpy import trapz

from torch import abs, sqrt, div, complex64, where, isinf, zeros_like, real, isnan
from torch.fft import fftshift
from torch.fft import fft2

from fastai.vision.all import AvgMetric
from monai.losses import SSIMLoss
from monai.metrics import PSNRMetric as _PSNR, RMSEMetric as _RMSE, MSEMetric as _MSE, MAEMetric as _MAE, SSIMMetric as _SSIM

from bioMONAI.core import torch_from_numpy

In [ ]:
#| export
def SSIMMetric(spatial_dims=3, **kwargs):
    def SSIM(x, y):
        return _SSIM(spatial_dims, **kwargs)(x,y).mean()
    return AvgMetric(SSIM)

def MSEMetric(**kwargs):
    def MSE(x, y):
        return _MSE(**kwargs)(x,y).mean()
    return AvgMetric(MSE)

def MAEMetric(**kwargs):
    def MAE(x, y):
        return _MAE(**kwargs)(x,y).mean()
    return AvgMetric(MAE)

def RMSEMetric(**kwargs):
    def RMSE(x, y):
        return _RMSE(**kwargs)(x,y).mean()
    return AvgMetric(RMSE)

def PSNRMetric(max_val, **kwargs):
    def PSNR(x, y):
        return _PSNR(max_val, **kwargs)(x,y).mean()
    return AvgMetric(PSNR)



### Fourier Ring Correlation

#### Radial mask

In [ ]:
#| export

def radial_mask(r,      # Radius of the radial mask
                cx=128, # X coordinate mask center
                cy=128, # Y coordinate maske center
                sx=256, # Size of the x-axis
                sy=256, # Size of the y-axis
                delta=1,# Thickness adjustment for the circular mask
               ):

    """
    Generate a radial mask.

    Returns:
       - numpy.ndarray: Radial mask.
    """
    
    sx = np.arange(0, sx)
    sy = np.arange(0, sy)

    # Calculate squared distances from each point in the grid to the center
    ind = (sx[np.newaxis, :] - cx) ** 2 + (sy[:, np.newaxis] - cy) ** 2

    # Define inner boundary of the circular mask
    ind1 = ind <= ((r[0] + delta) ** 2)

    # Define outer boundary of the circular mask
    ind2 = ind > (r[0] ** 2)

    # Create the radial mask by combining inner and outer boundaries
    return ind1 * ind2


In [ ]:
#| export

def get_radial_masks(width, # Width of the image
                     height, # Height of the image
                     ):

    """
    Generates a set of radial masks and corresponding to spatial frequencies.

    Returns:
        tuple: A tuple containing:
            - numpy.ndarray: Array of radial masks.
            - numpy.ndarray: Array of spatial frequencies corresponding to the masks.
    """

    # Calculate Nyquist frequency
    freq_nyq = int(np.floor(int(min(width, height)) / 2.0))
   
    # Generate radii from 0 to Nyquist frequency
    radii = np.arange(freq_nyq).reshape(freq_nyq, 1)

    # Generate radial masks using the radial_mask function    
    radial_masks = np.apply_along_axis(radial_mask, 1, radii, width//2, height//2, width, height, 1)

    # Calculate spatial frequencies
    spatial_freq = radii.astype(np.float32) / freq_nyq
    spatial_freq = spatial_freq / max(spatial_freq)
    spatial_freq = spatial_freq.squeeze(1)

    return radial_masks, spatial_freq


#### Fourier ring correlation

In [ ]:
#| export
def get_fourier_ring_correlations(image1, # First input image
                                  image2, # Second input image
                                  ):
  
    """
    Compute Fourier Ring Correlation (FRC) between two images.

    Returns:
        tuple: A tuple containing:
            - torch.Tensor: Fourier Ring Correlation values.
            - torch.Tensor: Array of spatial frequencies.
    """
    

    # Get image height and width
    height = image1.shape[len(image1.shape)-1]
    width = image1.shape[len(image1.shape)-2]
    
    # Get set of radial masks, spatial frequency, and Nyquist frequency
    radial_masks, spatial_frequency = get_radial_masks(height,width)

    # Get Nyquist frequency
    freq_nyq = len(spatial_frequency)
    
    # Transform tensor to complex
    image1 = image1.to(complex64)
    image2 = image2.to(complex64)

    # Transofrm array dimensions to (freq_nyq, width. height)
    image1 = image1.unsqueeze(0).repeat(freq_nyq, 1, 1)
    image2 = image2.unsqueeze(0).repeat(freq_nyq, 1, 1)

    # Convert spatial frequency and radial masks to torch.tensor
    spatial_frequency = torch_from_numpy(spatial_frequency)
    radial_masks = torch_from_numpy(radial_masks)

    # Transform tensor to complex
    radial_masks = radial_masks.to(complex64)
         
    # Compute fourier transform
    fft_image1 = fftshift(fft2(image1))
    fft_image2 = fftshift(fft2(image2))

    # Get elements only in the ring
    t1 = fft_image1 * radial_masks
    t2 = fft_image2 * radial_masks
        
    # image2 to complex conjugate
    t2_conj = t2.conj()

    # Numerator
    numerator = abs(real((t1 * t2_conj).sum(dim=(1,2))))

    # Denominator    
    denominator_1 = ((abs(t1) * abs(t1)).sum(dim=(1,2)))
    denominator_2 = ((abs(t2) * abs(t2)).sum(dim=(1,2)))     
    denominator = sqrt(denominator_1 * denominator_2)
   
    # Fourier shell correlation
    FRC = div(numerator, denominator)

    # Remove possible inf and NaN.
    FRC = where(isinf(FRC), zeros_like(FRC), FRC)  # inf
    FRC = where(isnan(FRC), zeros_like(FRC), FRC)  # NaN

    return FRC , spatial_frequency

In [ ]:
#| export

def FRCMetric(image1, # First input image
              image2, # Second input image
              ):


    """
    Compute the area under the Fourier Ring Correlation (FRC) curve between two images.

    Returns:
        - float: The area under the FRC curve.
    """

    # Calculate the Fourier Ring Correlation and spatial frequency
    FRC, spatial_frequency = get_fourier_ring_correlations(image1, image2)

    # Convert to numpy
    FRC = FRC.numpy()
    spatial_frequency = spatial_frequency.numpy()
      
    # Compute the area under the curve using trapezoidal integration
    area = trapz(FRC, spatial_frequency)
    
    return area


---

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()